In [181]:
from bs4 import BeautifulSoup
import requests
from lxml.html import fromstring

def getProxies(url):
    req = requests.get('https://free-proxy-list.net/')
    soup = BeautifulSoup(req.content,'lxml')
    rows = soup.find_all('tr')
    proxyList = []
    sp = 0
    for row in rows:
        if(row.findChild('td',class_='hx',recursive = False) != None):
            if(row.findChild('td',class_='hx',recursive = False).text == 'yes'):
                proxyList.append(row.td.text)
                proxyID = row.td.text
                try:
                    proxy = {
                        "http": 'http://1.4.221.231',
                        "https": 'http://' + proxyID       
                    }
                    r = requests.get('https://www.google.com/',proxies=proxy) 
                    sp = BeautifulSoup(r.content,'lxml')
                    System.out.println('This proxy worked: ' + proxyID)
                    return sp
                except:
                    print(row.td.text + ' Proxy didnt work')
def scrapeYelp(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'lxml')
    page_Divs = soup.find_all('div',class_='arrange_unit page-option')
    page_links = []
    page_links.append(url)
    for div in page_Divs:
        page_links.append(div.findChild('a')['href'])
    name_div = soup.find('div',class_='biz-page-header clearfix')
    end_div = name_div.findChild('div',class_='u-inline-block')
    currentRestaurant = name_div.findChild('h1',recursive = True).text + ' ' +end_div.findChild('h1').text
    ReviewDict = {}
    for i,link in enumerate(page_links):
        r = requests.get(link)
        soup = BeautifulSoup(r.content,'lxml')             
        for fullReview in soup.find_all('div',class_='review review--with-sidebar'):
            review = fullReview.findChild('div',class_='review-content',recursive=True)
            sidebar = fullReview.findChild('div', class_='review-sidebar',recursive=True)
            ReviewIDFull = sidebar.findChild('a',class_='arrange arrange--middle send-to-friend', recursive=True)['data-pop-uri']
            ReviewID = ReviewIDFull.replace('/send_to_friend/review/','')            
            ReviewDict[ReviewID] = ['Yelp',currentRestaurant]
            dateText = review.findChild('span', recursive = True).text
            dateText = dateText.replace('\n','')
            dateText = dateText.replace('Updated review','')
            dateText = dateText.strip()
            ReviewDict[ReviewID].append(dateText)
            reviewText = review.findChild('p',recursive=True).text
            ReviewDict[ReviewID].append(reviewText)
            
            ratingDiv = review.findChild('div',class_='biz-rating biz-rating-large clearfix')
            rating = ratingDiv.findChild('div',recursive=False).findChild('div',recursive=False)
            if(rating['title'] == '5.0 star rating'):
                ReviewDict[ReviewID].append(5)
            elif(rating['title'] == '4.0 star rating'):
                ReviewDict[ReviewID].append(4)
            elif(rating['title'] == '3.0 star rating'):
                ReviewDict[ReviewID].append(3)
            elif(rating['title'] == '2.0 star rating'):
                ReviewDict[ReviewID].append(2)
            else:
                ReviewDict[ReviewID].append(1)
            voteCount = 0
            voteDiv = fullReview.findChild('div',class_='rateReview voting-feedback',recursive=True)
            fullVote = voteDiv.findChildren('li',class_='vote-item inline-block',recursive=True)
            totalVotes = 0
            for vote in fullVote:
                voteType = vote.findChild('span',class_='vote-type',recursive=True).text
                if(voteType == 'Useful'):
                    voteCount = vote.findChild('span',class_='count',recursive=True).text
                    if(voteCount != ''):
                        totalVotes += int(voteCount)
                elif(voteType == 'Funny'):
                    voteCount = vote.findChild('span',class_='count',recursive=True).text
                    if(voteCount != ''):
                        totalVotes += int(voteCount)
                elif(voteType == 'Cool'):
                    voteCount = vote.findChild('span',class_='count',recursive=True).text
                    if(voteCount != ''):
                        totalVotes += int(voteCount)
            ReviewDict[ReviewID].append(totalVotes)
                
        
    print(ReviewDict)
scrapeYelp('https://www.yelp.com/biz/the-rook-ithaca')


{'YSq-MvZPGjecGgouEEEKsA': ['Yelp', 'The Rook', '10/21/2018', "We loved The Rook! Starting with the special appetizer fries set us off to a great start. They're spicy, salty, crunchy and delicious. Then we had two delicious entrees: the gnocchi and the salmon. My gnocchi was very autumnal- herbal and earthy with the root vegetables, yet fresh greens on top and complex flavors. And the salmon had a lovely crispy exterior with a soft center and nice farrow base to the dish. Then the dessert! We had the pecan pie- delicious. I'd definitely recommend. It was a nice warm atmosphere. Trendy vibe, cozy, nice staff. Note that you can call half an hour ahead on a busy night and get on a list so you don't have to wait for a table long when you arrive. Enjoy!", 5, 2], 'w5IMJfdvYoKoJYjlPRVKoQ': ['Yelp', 'The Rook', '10/20/2018', 'Such a cool small menu with combinations of veggies and meats and herbs I would never put together. Definitely get a cocktail from manny #bestbartender', 5, 0], 'ptVyJV-M